## Words and Tokens attention calculation

### 1) Loading weights

In [ ]:
import pandas as pd
import torch
import string
from transformers import AutoTokenizer

LABEL_MAPPER = [
    'antichina',
    'antivacina',
    'provacina',
]

CHECKPOINT = 'neuralmind/bert-base-portuguese-cased'
FOLDER = './results/'

attention_weights = torch.load(f'{FOLDER}/explanations.pt')
results_df = pd.read_csv(f'{FOLDER}/test_results_complete.csv', sep=';')

tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)

results_df['words'] = results_df['words'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
results_df['tokens'] = results_df['tokens'].apply(lambda x: x.split())

results_df

In [ ]:
# Just for testing...
for i, row in results_df.sample(5).iterrows():
    tokens = row.tokens
    weights = attention_weights[i]
    print(tokenizer.decode(tokenizer.convert_tokens_to_ids(tokens), skip_special_tokens=True))
    if len(tokens) != len(weights):
        print(f"Doesn't match on tweet: `{row.tweet}` ({len(tokens)} vs {len(weights)})")
    print('')

### 2) Save results counts...

In [ ]:
counts = {}

counts['antichina_correct'] = len(results_df.loc[(results_df.got == 0) & (results_df.expected == results_df.got)])
counts['antichina_wrong'] = len(results_df.loc[(results_df.got == 0) & (results_df.expected != results_df.got)])
counts['antichina'] = counts['antichina_correct'] + counts['antichina_wrong']

counts['antivacina_correct'] = len(results_df.loc[(results_df.got == 1) & (results_df.expected == results_df.got)])
counts['antivacina_wrong'] = len(results_df.loc[(results_df.got == 1) & (results_df.expected != results_df.got)])
counts['antivacina'] = counts['antivacina_correct'] + counts['antivacina_wrong']

counts['provacina_correct'] = len(results_df.loc[(results_df.got == 2) & (results_df.expected == results_df.got)])
counts['provacina_wrong'] = len(results_df.loc[(results_df.got == 2) & (results_df.expected != results_df.got)])
counts['provacina'] = counts['provacina_correct'] + counts['provacina_wrong']

counts['total'] = counts['antichina'] + counts['antivacina'] + counts['provacina']

counts_df = pd.DataFrame(
    columns = ['name', 'count'],
    data = [ (key, value) for key, value in counts.items() ]
)
counts_df.to_csv(f'{FOLDER}/result_counts.csv', index=None)
counts_df

In [ ]:
counts = {}

counts['antichina_antichina'] = len(results_df.loc[(results_df.expected == 0) & (results_df.got == 0)])
counts['antichina_antivacina'] = len(results_df.loc[(results_df.expected == 0) & (results_df.got == 1)])
counts['antichina_provacina'] = len(results_df.loc[(results_df.expected == 0) & (results_df.got == 2)])

counts['antivacina_antichina'] = len(results_df.loc[(results_df.expected == 1) & (results_df.got == 0)])
counts['antivacina_antivacina'] = len(results_df.loc[(results_df.expected == 1) & (results_df.got == 1)])
counts['antivacina_provacina'] = len(results_df.loc[(results_df.expected == 1) & (results_df.got == 2)])

counts['provacina_antichina'] = len(results_df.loc[(results_df.expected == 2) & (results_df.got == 0)])
counts['provacina_antivacina'] = len(results_df.loc[(results_df.expected == 2) & (results_df.got == 1)])
counts['provacina_provacina'] = len(results_df.loc[(results_df.expected == 2) & (results_df.got == 2)])

counts_df = pd.DataFrame(
    columns = ['name', 'count'],
    data = [ (key, value) for key, value in counts.items() ]
)
counts_df.to_csv(f'{FOLDER}/result_statistics.csv', index=None)
counts_df

### 3) Attentions

#### 3.1) Function to map tokens and weights to words
Calculates different attention weights for the words on each sentence
It employs:
- The tokens of each tweet
- The attention obtained by each token in the tweet

In [ ]:
import numpy as np

# Considerations:
# - A word can appear multiple times in the same tweet,
#   or in many tweets, it is going to be counted each
#   appearance.

def add_word_weight_to_mapping(
    words,
    word,
    tokens,
    weight,
    gotten,
    expected
):
    if word not in words:
        words[word] = {
            'tokens': ' '.join(tokens),
            'antichina_correct_w': 0.0,
            'antichina_correct_n': 0,
            'antichina_incorrect_w': 0.0,
            'antichina_incorrect_n': 0,

            'antivacina_correct_w': 0.0,
            'antivacina_correct_n': 0,
            'antivacina_incorrect_w': 0.0,
            'antivacina_incorrect_n': 0,

            'provacina_correct_w': 0.0,
            'provacina_correct_n': 0,
            'provacina_incorrect_w': 0.0,
            'provacina_incorrect_n': 0,
        }
    words[word][
        f"{LABEL_MAPPER[gotten]}_{'correct' if gotten == expected else 'incorrect'}_w"
    ] += weight
    words[word][
        f"{LABEL_MAPPER[gotten]}_{'correct' if gotten == expected else 'incorrect'}_n"
    ] += 1
    
    return words
    

def get_words_attentions(
    words_map,
    tokens,
    weights,
    gotten_label,
    expected_label
):
    current_sub_text_indices = []
    
    for i, token in enumerate(tokens):
        try:
            if token in tokenizer.all_special_tokens:
                continue

            if not token.startswith('##') and len(current_sub_text_indices):
                word = " ".join(
                    [tokens[idx] for idx in current_sub_text_indices]
                ).replace(" ##", "").strip()
                average_attention = np.mean([weights[idx] for idx in current_sub_text_indices])
                
                words_map = add_word_weight_to_mapping(
                    words_map,
                    word,
                    [tokens[idx] for idx in current_sub_text_indices],
                    average_attention,
                    gotten_label,
                    expected_label
                )
                
                current_sub_text_indices = []

            current_sub_text_indices.append(i)
        except:
            print(i, token)
            print(current_sub_text_indices)
            raise
        
        
    if len(current_sub_text_indices):
        word = " ".join(
                    [tokens[idx] for idx in current_sub_text_indices]
                ).replace(" ##", "").strip()
        average_attention = np.mean([weights[idx] for idx in current_sub_text_indices])
        
        words_map = add_word_weight_to_mapping(
            words_map,
            word,
            [tokens[idx] for idx in current_sub_text_indices],
            average_attention,
            gotten_label,
            expected_label
        )
        
    return words_map

#### 3.2) Function to map weights to tokens
Calculates different attention weights for the tokens on each sentence
It employs:
- The attention obtained by each token in the tweet

In [ ]:
import numpy as np

# Considerations:
# - A token can appear multiple times in the same tweet,
#   or in many tweets, it is going to be counted each
#   time it appears.

def add_token_weight_to_mapping(
    tokens,
    token,
    weight,
    gotten,
    expected
):
    if token not in tokens:
        tokens[token] = {
            'antichina_correct_w': 0.0,
            'antichina_correct_n': 0,
            'antichina_incorrect_w': 0.0,
            'antichina_incorrect_n': 0,

            'antivacina_correct_w': 0.0,
            'antivacina_correct_n': 0,
            'antivacina_incorrect_w': 0.0,
            'antivacina_incorrect_n': 0,

            'provacina_correct_w': 0.0,
            'provacina_correct_n': 0,
            'provacina_incorrect_w': 0.0,
            'provacina_incorrect_n': 0,
        }
    tokens[token][
        f"{LABEL_MAPPER[gotten]}_{'correct' if gotten == expected else 'incorrect'}_w"
    ] += weight
    tokens[token][
        f"{LABEL_MAPPER[gotten]}_{'correct' if gotten == expected else 'incorrect'}_n"
    ] += 1
    
    return tokens
    

def get_tokens_attentions(
    tokens_map,
    tokens,
    weights,
    gotten_label,
    expected_label
):
    current_sub_text_indices = []
    
    for i, token in enumerate(tokens):
        try:
            if token in tokenizer.all_special_tokens:
                continue
                
            tokens_map = add_token_weight_to_mapping(
                tokens_map,
                token,
                weights[i],
                gotten_label,
                expected_label
            )
        except:
            print(i, token)
            raise        

    return tokens_map

#### 3.3) Caculations...

In [ ]:
words_map = {}
tokens_map = {}

for i, row in results_df.iterrows():
    try:
        words_map = get_words_attentions(
            words_map,
            row.tokens,
            attention_weights[i],
            row.got,
            row.expected
        )
        
        tokens_map = get_tokens_attentions(
            tokens_map,
            row.tokens,
            attention_weights[i],
            row.got,
            row.expected
        )
    except:
        print(f'Error found in {i}: {row.tweet}')
        print(len(row.tokens), len(attention_weights[i]))
        print(row.tokens)
        print(attention_weights[i])
        print('')
        raise
        break

#### 3.4) Save words attentions

In [ ]:
words_df = pd.DataFrame(words_map).T.sort_index()
words_df.insert(0, column='word', value=words_df.index)
words_df.to_csv(f'{FOLDER}/words_attention.csv', index=None, sep=';')
words_df

#### 3.5) Save tokens attentions

In [ ]:
tokens_df = pd.DataFrame(tokens_map).T.sort_index()
tokens_df.insert(0, column='token', value=tokens_df.index)
tokens_df.to_csv(f'{FOLDER}/tokens_attention.csv', index=None, sep=';')
tokens_df

### 4) TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

STOPWORDS = nltk.corpus.stopwords.words('portuguese') + list(string.punctuation)

vectorizer = TfidfVectorizer(
#     strip_accents='ascii', # This is OPTIONAL
    stop_words=STOPWORDS,
    use_idf=True
)

vectors = vectorizer.fit_transform(results_df.words.values)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()

tf_idf = pd.DataFrame(
    denselist,
    columns=feature_names
).T

# Save TF-IDF weights for each word
tf_idf.to_csv(f'{FOLDER}/tfidf_weights.csv', sep=';')

In [ ]:
# Save the indices of tweets of each group
antichina = results_df[results_df.expected == 0]
antivacina = results_df[results_df.expected == 1]
provacina = results_df[results_df.expected == 2]

indices = {
    'antichina': {
        'all': list(antichina.index),
        'correct': list(antichina[antichina.got == antichina.expected].index),
        'wrong': list(antichina[antichina.got != antichina.expected].index),
    },
    'antivacina': {
        'all': list(antivacina.index),
        'correct': list(antivacina[antivacina.got == antivacina.expected].index),
        'wrong': list(antivacina[antivacina.got != antivacina.expected].index),
    },
    'provacina': {
        'all': list(provacina.index),
        'correct': list(provacina[provacina.got == provacina.expected].index),
        'wrong': list(provacina[provacina.got != provacina.expected].index),
    },
}

indices_df = pd.DataFrame(indices)
indices_df.insert(0, column='group', value=indices_df.index)
indices_df.to_csv(f'{FOLDER}/tfidf_indices.csv', sep=';', index=None)
indices_df